### Code that record servo values after moving the arm manually

In [ ]:
from Arm_Lib import Arm_Device
import time, json

arm = Arm_Device()
time.sleep(0.2)
print("DOFBOT connected")

# Optionally disable torque so you can manually or with joystick reposition
arm.Arm_serial_set_torque(0)
print("Torque off ï¿½ arm is free to move manually (joystick / by hand)")

def read_pose():
    return [arm.Arm_serial_servo_read(i) for i in range(1, 7)]

coords = []
for i in range(1, 5):
    input(f"Move arm to POINT {i}, then press ENTER ? ")
    pose = read_pose()
    print("Recorded angles:", pose)
    coords.append(pose)

with open("dofbot_pi_poses.json", "w") as f:
    json.dump(coords, f)

print("Saved poses:", coords)

# To replay ï¿½ re-enable torque, then write
arm.Arm_serial_set_torque(1)
time.sleep(0.2)

for p in coords:
    arm.Arm_serial_servo_write6(*p, 1500)  # 1.5s movement
    time.sleep(2)

print("Replay done")

### Code used to manually map every squares from a1 to h8

In [ ]:
from Arm_Lib import Arm_Device
import json, time, pathlib

arm = Arm_Device()

print("=== TEACH EVERY SQUARE BY TOUCH ===")
print("Move arm so gripper tip JUST touches the square, then press ENTER.")

board = {}         

for rank in range(1, 9):            # 1..8
    for file in range(ord('a'), ord('h')+1):   # a..h
        sq = chr(file) + str(rank)
        input(f"? {sq}  (press ENTER when touching)")
        angles = [arm.Arm_serial_servo_read(i) for i in range(1, 7)]
        board[sq] = angles + [0]      # z = 0 (touch reference) Note: was 5 before but proved to be causing issues so changed to 0
        print(f"  recorded {sq}: {angles}")

# save
pathlib.Path("/media/all_squares_touch.json").write_text(json.dumps(board, indent=2))
print("? Saved ? /media/all_squares_touch.json")